<a href="https://colab.research.google.com/github/EngRenan/Regras_Associacao/blob/main/Regras_associacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Regras de Associação**

Dentro do tipo de aprendizado supervisionado temos o algoritmo de regras de associação no qual seu principal objetivo é identificar padrões relevantes nos dados, que possam ser usados para ajudar a definir estratégias para o negocio ou identificar um tipo de comportamento. 

Neste arquivo iremos importar uma base de dados ficticio para demonstrar como é realizado, estruturado, montado um algoritmo de regras de associacao

In [ ]:
# Instalando bibliotecas necessarias
!pip install mlxtend
!pip install pandas

In [5]:
# Importando as bibliotecas
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

In [9]:
# Selecionando o dataset
dados_mercado = pd.read_csv('/content/mercado.csv', sep=';') # Utilizado o sep=';' para realizar a separação atraves dos ; existentes no arquivo csv

# Exibindo dados 
dados_mercado

,produto1,produto2,produto3,produto4
0,leite,cerveja,pao,manteiga
1,pao,cafe,manteiga,NaN
2,pao,cafe,manteiga,NaN
3,leite,cafe,pao,manteiga
4,cerveja,NaN,NaN,NaN
5,manteiga,NaN,NaN,NaN
6,pao,NaN,NaN,NaN
7,feijao,NaN,NaN,NaN
8,arroz,feijao,NaN,NaN
9,arroz,NaN,NaN,NaN


Observe que os arquivos que tem valores nulos sao representados por NaN. Neste caso o algoritmo de regras de associacao necessita que o input de dados seja em formato de lista contendo indicador de true or false para cada produto. Necessario tratar o dataset para satisfazer a premissa. 

In [10]:
# Visualizando tamanho do dataset
len(dados_mercado)

10

In [11]:
# Acessando o produto dentro da lista
print(dados_mercado.values[1,1]) # Linha 0 coluna 3 

cafe


In [14]:
# Acessar cada item dentro do dataframe e inserir em uma lista 
lista_compras = []
for i in range(len(dados_mercado)):
  lista_compras.append([str(dados_mercado.values[i, j]) for j in range (4)])

# Visualizando a lista de compras gerada
lista_compras

[['leite', 'cerveja', 'pao', 'manteiga'],
 ['pao', 'cafe', 'manteiga', 'nan'],
 ['pao', 'cafe', 'manteiga', 'nan'],
 ['leite', 'cafe', 'pao', 'manteiga'],
 ['cerveja', 'nan', 'nan', 'nan'],
 ['manteiga', 'nan', 'nan', 'nan'],
 ['pao', 'nan', 'nan', 'nan'],
 ['feijao', 'nan', 'nan', 'nan'],
 ['arroz', 'feijao', 'nan', 'nan'],
 ['arroz', 'nan', 'nan', 'nan']]

Agora sera necessario criar uma lista de produtos com o indicador de true or false, conforme descrito linhas acima. 

In [15]:
te = TransactionEncoder()
te_array = te.fit(lista_compras).transform(lista_compras)

# Visualizando a lista de produtos criada
te_array

array([[False, False,  True, False,  True,  True, False,  True],
       [False,  True, False, False, False,  True,  True,  True],
       [False,  True, False, False, False,  True,  True,  True],
       [False,  True, False, False,  True,  True, False,  True],
       [False, False,  True, False, False, False,  True, False],
       [False, False, False, False, False,  True,  True, False],
       [False, False, False, False, False, False,  True,  True],
       [False, False, False,  True, False, False,  True, False],
       [ True, False, False,  True, False, False,  True, False],
       [ True, False, False, False, False, False,  True, False]])

**Criando um dataframe com is itens do mercado**.
Criacao de um dataframe atraves da lista criada e informando as colunas que reprensentam os itens do dataset

In [20]:
df_prod_mercado = pd.DataFrame(te_array, columns= te.columns_)

# Visualizando o dataframe
df_prod_mercado

,arroz,cafe,cerveja,feijao,leite,manteiga,nan,pao
0,False,False,True,False,True,True,False,True
1,False,True,False,False,False,True,True,True
2,False,True,False,False,False,True,True,True
3,False,True,False,False,True,True,False,True
4,False,False,True,False,False,False,True,False
5,False,False,False,False,False,True,True,False
6,False,False,False,False,False,False,True,True
7,False,False,False,True,False,False,True,False
8,True,False,False,True,False,False,True,False
9,True,False,False,False,False,False,True,False


In [21]:
# Excluindo a coluna NAN do dataframe para que nao haja contaminação dos dados, pois essa coluna é gerada pelos valores ausentes visto anteriormente como NaN
df_prod_mercado.drop(columns='nan', inplace=True)

# Visualizando novamente sem a coluna
df_prod_mercado

,arroz,cafe,cerveja,feijao,leite,manteiga,pao
0,False,False,True,False,True,True,True
1,False,True,False,False,False,True,True
2,False,True,False,False,False,True,True
3,False,True,False,False,True,True,True
4,False,False,True,False,False,False,False
5,False,False,False,False,False,True,False
6,False,False,False,False,False,False,True
7,False,False,False,True,False,False,False
8,True,False,False,True,False,False,False
9,True,False,False,False,False,False,False


Gerando os itens mais frequentes do dataframe utilizando a biblioteca apriori

In [23]:
from mlxtend.frequent_patterns import apriori

In [24]:
frequent_itemsets = apriori(df_prod_mercado, min_support=0.3, use_colnames=True) # Seleciona apenas suporte de 0,4
frequent_itemsets

# Selecionado os itens sets com o suporte de 30% nesse experimento

,support,itemsets
0,0.3,(cafe)
1,0.5,(manteiga)
2,0.5,(pao)
3,0.3,"(manteiga, cafe)"
4,0.3,"(cafe, pao)"
5,0.4,"(manteiga, pao)"
6,0.3,"(manteiga, cafe, pao)"


Criando as regras de associacao importando a biblioteca **association_rules** para gerar as regras. 

---



In [25]:
from mlxtend.frequent_patterns import association_rules

In [26]:
regras = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8) # 80% regras acontecerem

# Visualizando as regras criadas
regras

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(cafe),(manteiga),0.3,0.5,0.3,1.0,2.0,0.15,inf
1,(cafe),(pao),0.3,0.5,0.3,1.0,2.0,0.15,inf
2,(manteiga),(pao),0.5,0.5,0.4,0.8,1.6,0.15,2.5
3,(pao),(manteiga),0.5,0.5,0.4,0.8,1.6,0.15,2.5
4,"(manteiga, cafe)",(pao),0.3,0.5,0.3,1.0,2.0,0.15,inf
5,"(cafe, pao)",(manteiga),0.3,0.5,0.3,1.0,2.0,0.15,inf
6,(cafe),"(manteiga, pao)",0.3,0.4,0.3,1.0,2.5,0.18,inf


In [27]:
# Selecionando colunas para analisar 
regras = regras[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

# Renomeando colunas 
regras = regras.rename(columns={"antecedents": "se", "consequents": "então",
                                "support": "suporte",
                                "confidence": "confiança", "lift": "LIFT"})

# Visualizando 
regras 

,se,então,suporte,confiança,LIFT
0,(cafe),(manteiga),0.3,1.0,2.0
1,(cafe),(pao),0.3,1.0,2.0
2,(manteiga),(pao),0.4,0.8,1.6
3,(pao),(manteiga),0.4,0.8,1.6
4,"(manteiga, cafe)",(pao),0.3,1.0,2.0
5,"(cafe, pao)",(manteiga),0.3,1.0,2.0
6,(cafe),"(manteiga, pao)",0.3,1.0,2.5


Filtrando as melhores regras baseado no LIFT

In [28]:
regras.sort_values(by = 'LIFT', ascending = False)

,se,então,suporte,confiança,LIFT
6,(cafe),"(manteiga, pao)",0.3,1.0,2.5
0,(cafe),(manteiga),0.3,1.0,2.0
1,(cafe),(pao),0.3,1.0,2.0
4,"(manteiga, cafe)",(pao),0.3,1.0,2.0
5,"(cafe, pao)",(manteiga),0.3,1.0,2.0
2,(manteiga),(pao),0.4,0.8,1.6
3,(pao),(manteiga),0.4,0.8,1.6


A tabela demonstrada acima descreve que: 



1.   Clientes que compram café tem 2,5 vezes mais chances de comprarem manteiga e pão
2.   Em 100% dos casos, clientes que compram café também compram manteiga e pão 

Com base nesses resultados, ajudaria muito uma empresa do setor alimenticio ou rede de supoermercados implantar tecnicas e tomar decisões para melhores vendas com base no que os dados trouxeram. 

